In [1]:
import torch
from torch import nn
from model import TextRNN
from cnews_loader import read_vocab,read_category,process_file
import numpy as np
import torch.utils.data as Data
from torch import optim

/usr/local/miniconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/miniconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/miniconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/miniconda3/envs/dl/lib/python3.7/site-packages/ten

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [2]:
#设置训练集
train_file='cnews.train.small.txt'
#设置测试集
test_file='cnews.test.txt'
#设置验证集
val_file='cnews.val.txt'
#设置词汇集
vocab_file='cnews.vocab.txt'

In [3]:
def train():
    #定义RNN模型
    MODEL=TextRNN().cuda()
    #定义损失函数
    Loss=nn.MultiLabelSoftMarginLoss()
    #设置优化器，学习率lr=0.001
    optimizer=optim.Adam(MODEL.parameters(),lr=0.001)
    #定义best_model_acc保存最好的模型,初始值设为0
    best_model_acc=0
    #开始训练，迭代次数1000次
    for epoch in range(1000):
        print('epoch=',epoch)
        #分批训练
        for step,(X_batch,Y_batch) in enumerate(train_loader):
            x=X_batch.cuda()
            y=Y_batch.cuda()
            #前向传播得出模型输出
            out=MODEL(x)
            #计算损失函数
            loss=Loss(out,y)
            print('loss=',loss)
            #反向传播
            optimizer.zero_grad()#梯度清零
            optimizer.step()
            #计算准确率
            accuracy=np.mean((torch.argmax(out,1)==torch.argmax(y,1)).cpu().numpy())
            print('accuracy',accuracy)
       
        #每迭代训练5次进行一次验证，
        if (epoch+1)%5==0:
            for step,(X_batch,Y_batch) in enumerate(val_loader):
                print("step=",step)
                x=X_batch.cuda()
                y=Y_batch.cuda()
                #前向传播
                out=MODEL(x)
                #计算准确率
                accuracy=np.mean((torch.argmax(out,1)==torch.argmax(y,1)).cpu().numpy())
                #如何当前的准确率高于已知准确率，则更新并保存当前模型
                if accuracy>best_model_acc:
                    torch.save(MODEL,'best_model.pkl')
                    best_model_acc=accuracy
                    print('best_model.pkl saved')
                    print('accuracy=',accuracy)

In [19]:
#数据预处理
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
print(categories)
# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab(vocab_file)
print(words)
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file(train_file, word_to_id, cat_to_id, 600)
print('x_train=', x_train)
x_val, y_val = process_file(val_file, word_to_id, cat_to_id, 600)


['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
['<PAD>', '，', '的', '。', '一', '是', '在', '0', '有', '不', '了', '中', '1', '人', '大', '、', '国', '', '2', '这', '上', '为', '个', '“', '”', '年', '学', '时', '我', '地', '和', '以', '到', '出', '来', '会', '行', '发', '：', '对', '们', '要', '生', '家', '他', '能', '也', '业', '金', '3', '成', '可', '分', '多', '现', '5', '就', '场', '新', '后', '于', '下', '日', '经', '市', '前', '过', '方', '得', '作', '月', '最', '开', '房', '》', '《', '高', '9', '8', '.', '而', '比', '公', '4', '说', ')', '将', '(', '都', '资', 'e', '6', '基', '用', '面', '产', '还', '自', '者', '本', '之', '美', '很', '同', '', '7', '部', '进', '但', '主', '外', '动', '机', '元', '理', '加', 'a', '全', '与', '实', '影', '好', '小', '间', '其', '天', '定', '表', '力', '如', '次', '合', '长', 'o', '体', '价', 'i', '所', '内', '子', '目', '电', '-', '当', '度', '品', '看', '期', '关', '更', 'n', '等', '工', '然', '斯', '重', '些', '球', '此', '里', '利', '相', '情', '投', '点', '没', '因', '已', '三', '心', '特', '明', '位', '两', '法', '从', '入', '名', '万', '手', '计', '性', '事', '只', '样', '示', 'r', '

x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [   0    0    0 ...  701   35    3]
 [ 360   73    2 ...  695  450    3]
 [   0    0    0 ... 1537  637   85]]
x_test [[ 426   28   40 ...  116  599   85]
 [  20    2 1397 ... 3246 1596   85]
 [   0    0    0 ...  134  409   85]
 ...
 [   0    0    0 ...  221  365    3]
 [ 431   50  371 ...  148  483    3]
 [   0    0    0 ...  993 2577 1072]]


In [5]:
#设置GPU
cuda=torch.device('cuda')
#设置批次大小
Batch_size=128
#将训练数据转换为Tensor格式
x_train,y_train=torch.LongTensor(x_train),torch.Tensor(y_train)
#将验证数据转换为Tensor格式
x_val,y_val=torch.LongTensor(x_val),torch.Tensor(y_val)

train_dataset=Data.TensorDataset(x_train,y_train)
train_loader=Data.DataLoader(dataset=train_dataset,batch_size=Batch_size,shuffle=True)#随机打乱数据

val_dataset=Data.TensorDataset(x_val,y_val)
val_loader=Data.DataLoader(dataset=val_dataset,batch_size=Batch_size)
#调用训练函数，进行模型训练
train()

epoch= 0
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7341, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1328125
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1796875
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.109375
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09615384615384616
epoch= 1


/usr/local/miniconda3/envs/dl/lib/python3.7/site-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1328125
loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7348, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09615384615384616
epoch= 2
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss= tens

loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7348, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.11538461538461539
epoch= 13
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1484375
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1640625
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1328125
loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09375
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1346153846153846

loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7341, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.11538461538461539
step= 0
step= 1
step= 2
step= 3
step= 4
step= 5
step= 6
step= 7
step= 8
step= 9
step= 10
step= 11
step= 12
step= 13
step= 14
step= 15
step= 16
step= 17
step= 18
step= 19
step= 20
step= 21
step= 22
step= 23
step= 24
step= 25
step= 26
step= 27
step= 28
step= 29
step= 30
step= 31
step= 32
step= 33
step= 34
step= 35
step= 36
step= 37
step= 38
step= 39
epoch= 25
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0546875
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.140625
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackwa

...

loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09375
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0673076923076923
epoch= 947
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7342, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09375
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.08653846153846154
epoch= 948
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
ac

loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09615384615384616
epoch= 959
loss= tensor(0.7347, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1328125
loss= tensor(0.7348, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09615384615384616
step= 0
step= 1
step= 2
step= 3
step= 4
step= 5
step= 6
step= 7
step= 8
step= 9
step= 10
step= 11
step= 12
step= 13
step= 14
step= 15
step= 16
step= 17
step= 18
step= 19
step= 20
step= 21
step= 22
step= 23
step= 24
step= 25
step= 26
step

step= 10
step= 11
step= 12
step= 13
step= 14
step= 15
step= 16
step= 17
step= 18
step= 19
step= 20
step= 21
step= 22
step= 23
step= 24
step= 25
step= 26
step= 27
step= 28
step= 29
step= 30
step= 31
step= 32
step= 33
step= 34
step= 35
step= 36
step= 37
step= 38
step= 39
epoch= 970
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.140625
loss= tensor(0.7341, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1875
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7350, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1346153846153846
epoch= 971
loss= tensor(0.7346, device='cuda:0', grad_fn=<M

loss= tensor(0.7348, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.109375
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.10576923076923077
epoch= 982
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.109375
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0546875
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.09375
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.109375
los

loss= tensor(0.7348, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7348, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.078125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.10576923076923077
epoch= 994
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss= tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0625
loss= tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss= tensor(0.7341, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1328125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.03125
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.109375
loss= tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1328125
loss= tensor(0.7345, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0865384615

In [23]:
#加载测试集,用测试集测试
x_test,y_test=process_file(test_file, word_to_id, cat_to_id, 600)
#rint('x_test',x_test)
#将测试数据转换为Tensor格式
x_test,y_test=torch.LongTensor(x_test),torch.Tensor(y_test)
test_dataset=Data.TensorDataset(x_test,y_test)
test_loader=Data.DataLoader(dataset=test_dataset,batch_size=Batch_size)
best_test_acc=0

for step,(X_batch,Y_batch) in enumerate(test_loader):
    x_test=X_batch.cuda()
    y_test=Y_batch.cuda()
    #前向传播
    MODEL=torch.load('best_model.pkl')
    out_test=MODEL(x_test)
    #计算准确率
    accuracy_test=np.mean((torch.argmax(out_test,1)==torch.argmax(y_test,1)).cpu().numpy())
    #如何当前的准确率高于已知准确率，则更新并保存当前模型
    if accuracy_test>best_test_acc:
        best_test_acc=accuracy_test
print('训练后的模型用测试集测试，最高准确率为：',best_test_acc)

训练后的模型用测试集测试，最高准确率为： 0.2421875
